In [27]:
import dotenv
import os
print("os.getenv('zilliqa_uri'):", os.getenv("ZILLIZ_URI"))


os.getenv('zilliqa_uri'): None


In [ ]:
import requests
import json
session=requests.Session()
login_payload={
    "email":"mukkawarshravan04@gmail.com",
    "password":"2250213"
}
login_res=session.post("http://localhost:5000/api/v1/users/signIn",json=login_payload)
cookies=session.cookies.get_dict()

groups=session.get("http://localhost:5000/api/v1/group/getG")
# print(groups.json())
print(json.dumps(groups.json()))

{'statusCode': 200, 'data': [{'_id': '68566e3e594730eadfd4c0c1', 'name': '407', 'leader': {'_id': '68566cb3594730eadfd4bdde', 'fullName': 'John'}, 'members': ['68566cb3594730eadfd4bdde', '68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'hostel', 'whiteboard': '68566e3e594730eadfd4c0bf', 'createdAt': '2025-06-21T08:33:02.126Z', '__v': 0}, {'_id': '6857f71ca29cb235823a2ab9', 'name': 'Shravan mukkawar', 'leader': {'_id': '68566cafbe81f4c113ca432c', 'fullName': 'Shravan Mukkawar'}, 'members': ['68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'Main group', 'tags': 'React', 'category': 'Web Development', 'whiteboard': '6857f71ca29cb235823a2ab7', 'createdAt': '2025-06-22T12:29:16.860Z', '__v': 0}, {'_id': '68580dee735f02eabc6c4a65', 'name': 'Prompt Engineers', 'leader': {'_id': '68566cafbe81f4c113ca432c', 'fullName': 'Shravan Mukkawar'}, 'members': ['68566cafbe81f4c113ca432c'], 'resources': [], 'description': 'Work on LangChain, LLMs and smart agents', 'tags': 'LangCha

In [ ]:
import json

# Get JSON from response
data = groups.json()

# Get the list of groups
groups_list = data.get("data", [])

# Open a text file to write the output
with open("groups_info.txt", "w", encoding="utf-8") as f:
    for i, group in enumerate(groups_list, start=1):
        name = group.get("name", "N/A")
        description = group.get("description", "N/A")
        category = group.get("category", "N/A")
        tags = group.get("tags", "N/A")

        f.write(f"Group {i}:\n")
        f.write(f"Name       : {name}\n")
        f.write(f"Description: {description}\n")
        f.write(f"Category   : {category}\n")
        f.write(f"Tags       : {tags}\n")
        f.write("-" * 40 + "\n")

print("Group info saved to groups_info.txt")


NameError: name 'groups' is not defined

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

llm=Ollama(model='llama2:7b')


C:\Users\mukka\AppData\Local\Temp\ipykernel_30820\1620002507.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model='llama2:7b')


In [3]:
groups = []

with open("groups_info.txt", "r", encoding="utf-8") as f:
    content = f.read().split("----------------------------------------\n")
    for block in content:
        if block.strip():
            groups.append(block.strip())

print(groups)

['Group 1:\nName       : 407\nDescription: hostel\nCategory   : N/A\nTags       : N/A', 'Group 2:\nName       : Shravan mukkawar\nDescription: Main group\nCategory   : Web Development\nTags       : React', 'Group 3:\nName       : Prompt Engineers\nDescription: Work on LangChain, LLMs and smart agents\nCategory   : AI & ML\nTags       : LangChain,LLM,Agents', 'Group 4:\nName       : React Masters\nDescription: Deep dive into React, hooks, context, and advanced patterns.\nCategory   : Web Development\nTags       : React, JavaScript, Frontend', 'Group 5:\nName       : LangChain Coders\nDescription: Discuss advanced LangChain projects and agents\nCategory   : AI & ML\nTags       : LangChain,LLMs,Embeddings', 'Group 6:\nName       : Arduino Hackers\nDescription: Tinker with sensors and microcontrollers.\nCategory   : Embedded Systems\nTags       : Arduino, IoT, Electronics', 'Group 7:\nName       : Generative AI Creators Hub\nDescription: A community for exploring, experimenting with, and b

In [4]:
from langchain.schema import Document

documents = []

for group in groups:
    lines = group.splitlines()
    metadata = {}
    content_parts = []

    for line in lines:
        if line.startswith("Name"):
            metadata["name"] = line.split(":", 1)[1].strip()
        elif line.startswith("Description"):
            description = line.split(":", 1)[1].strip()
            content_parts.append(description)
        elif line.startswith("Category"):
            category = line.split(":", 1)[1].strip()
            content_parts.append(category)
        elif line.startswith("Tags"):
            tags_str = line.split(":", 1)[1].strip()
            tags = [tag.strip() for tag in tags_str.split(",")]
            content_parts.extend(tags)

    merged_content = " ".join(content_parts)

    documents.append(Document(page_content=merged_content, metadata=metadata))


In [5]:
embedding_model = OllamaEmbeddings(model='llama2:7b')


C:\Users\mukka\AppData\Local\Temp\ipykernel_30820\3346639310.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model='llama2:7b')


In [6]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="example_collection",
    persist_directory="./chroma_langchain_db"
)

vectorstore.persist()


C:\Users\mukka\AppData\Local\Temp\ipykernel_30820\532723675.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [7]:
# Dummy query to fetch everything with very low threshold
all_docs = vectorstore.similarity_search("", k=vectorstore._collection.count())

for i, doc in enumerate(all_docs):
    print(f"\n--- Document {i+1} ---")
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)



--- Document 1 ---
Content: A group for voice recognition and synthesis enthusiasts. Explore ASR, TTS, voice cloning, and multilingual speech models. AI & ML VoiceAI ASR TTS SpeechRecognition VoiceCloning MultilingualAI
Metadata: {'name': 'Voice AI & Speech Tech'}

--- Document 2 ---
Content: Work on LangChain, LLMs and smart agents AI & ML LangChain LLM Agents
Metadata: {'name': 'Prompt Engineers'}

--- Document 3 ---
Content: Discuss advanced LangChain projects and agents AI & ML LangChain LLMs Embeddings
Metadata: {'name': 'LangChain Coders'}

--- Document 4 ---
Content: A community for computational biologists and geneticists working on DNA sequencing, genome assembly, and data-driven biological discovery using algorithms and software tools. Healthcare & Biotech Bioinformatics Genomics Sequencing CRISPR ComputationalBiology DataScience
Metadata: {'name': 'Bioinformatics & Genomic Analysis'}

--- Document 5 ---
Content: For engineers deploying machine learning on real-time streams.

In [10]:
vectorstore = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db"
)

retriever = vectorstore.as_retriever(search_kwargs={"k":3})
query = "python"
docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(docs):
    print(f"\n--- Retriever Result {i+1} ---")
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)


    



--- Retriever Result 1 ---
Content: Discuss advanced LangChain projects and agents AI & ML LangChain LLMs Embeddings
Metadata: {'name': 'LangChain Coders'}

--- Retriever Result 2 ---
Content: Main group Web Development React
Metadata: {'name': 'Shravan mukkawar'}

--- Retriever Result 3 ---
Content: Work on LangChain, LLMs and smart agents AI & ML LangChain LLM Agents
Metadata: {'name': 'Prompt Engineers'}


In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Qdrant
from langchain.schema import Document
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(host="localhost", port=6333)

collection_name = "zilliz_llama_test"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE)  
)

embedding_model = OllamaEmbeddings(model='llama2:7b')


vectorstore = Qdrant.from_documents(
    documents=documents,
    embedding=embedding_model,
    url="http://localhost:6333",
    collection_name=collection_name
)




C:\Users\mukka\AppData\Local\Temp\ipykernel_19764\2279238761.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [28]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a strict assistant. From the given context, identify the **five most relevant groups ma,es** for the user's question.
If there are groups which are not relevant remove it from the list.
Each group is described with fields like "Name" only.

Your task is to:
- Extract and return only the **group names**
- Do not include any explanation or extra text

Return the group names as a numbered list.

Context:
{context}

Question:
{question}

Return format:
1. <Group Name 1>  
2. <Group Name 2>  
3. <Group Name 3>  
4. <Group Name 4>  
5. <Group Name 5>
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [29]:
query = "Blockchain"
response = qa_chain.run(query)

print("Recommended Group:", response)


Recommended Group: Based on the given context, the five most relevant groups for the user's question are:

1. Blockchain Developers
2. Cryptocurrency Enthusiasts
3. Decentralized Applications (dApps)
4. Smart Contract Developers
5. Blockchain Researchers


In [1]:
# connection_args = {
#     "uri": os.getenv("ZILLIZ_URI"),
#     "user": os.getenv("ZILLIZ_USER"),
#     "password": os.getenv("ZILLIZ_PASSWORD"),
#     "secure": True
# }

In [19]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Step 3: Format the prompt manually
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a strict assistant. From the given context, identify the **five most relevant groups** for the user's question.
use groups descriptions,category,tags to answer the question.
If there are groups which are not relevant remove it from the list.
Each group is described with fields like "Name       :", "Description:", etc.

Your task is to:
- Extract and return only the **group names**
- **Exactly as they appear** in the "Name       :" field
- Do not include any explanation or extra text

Return the group names as a numbered list.

Context:
{context}

Question:
{question}

Return format:
1. <Group Name 1>  
2. <Group Name 2>  
3. <Group Name 3>  
4. <Group Name 4>  
5. <Group Name 5>
"""
)

# Step 4: Create a simple LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

context=groups
response = llm_chain.run({
    "context": context,
    "question": "Which groups are related to Blockchain?"
})

print(response)


None of the groups in the list are directly related to blockchain technology. However, some groups may have indirect connections to blockchain through their focus on emerging technologies or digital innovation. Here are five groups from the list that could potentially have a connection to blockchain:

1. <Group Name 62> Quantum Computing & Cryptography Discussion Forum - This group focuses on the intersection of quantum computing and cryptography, which is an area of research that has potential applications in blockchain technology.
2. <Group Name 79> Artificial Intelligence & Machine Learning Community - AI and machine learning are key technologies in the development of blockchain platforms, and this group could provide insights into the latest developments in these fields.
3. <Group Name 85> Renaissance Art Restoration Club - While not directly related to blockchain, this group could provide insights into the preservation and restoration of cultural heritage, which could have implica

In [16]:
!pip install -qU "langchain-chroma>=0.1.2"